In [7]:
from mcx_ultrasound_opsbased import MCX
import calculateR_CV
import json
import os
import numpy as np
import pandas as pd
from glob import glob
from time import sleep
from tqdm import tqdm
import time
import sys

In [8]:
# %% run

class Timer():
    def __init__(self):
        self.o = time.time()

    def measure(self, p=1):
        x = (time.time() - self.o) / p
        x = int(x)
        if x >= 3600:
            return '{:.1f}h'.format(x / 3600)
        if x >= 60:
            return '{}m'.format(round(x / 60))
        return '{}s'.format(x)


def run_mcx(result_folder, mus_type, subject, mus_start, mus_end, NA_enable, NA, runningNum, cvThreshold, repeatTimes, ijv_type):
    mus_set = np.load(os.path.join("OPs_used", f"{mus_type}_mus_set.npy"))
    timer = Timer()
    ID = f'{subject}_ijv_{ijv_type}'
    for run_idx in tqdm(range(mus_start, mus_end+1)):
        now = time.time()
        #  Setting
        session = f"run_{run_idx}"
        sessionID = os.path.join("result", result_folder, ID, mus_type, session)
        
        #  Do simulation
        # initialize
        simulator = MCX(sessionID)

        with open(os.path.join(sessionID, "config.json")) as f:
            config = json.load(f)
        simulationResultPath = os.path.join(
            config["OutputPath"], session, "post_analysis", f"{session}_simulation_result.json")

        with open(simulationResultPath) as f:
            simulationResult = json.load(f)
        existedOutputNum = simulationResult["RawSampleNum"]
        # run forward mcx
        if runningNum:
            for idx in range(existedOutputNum, existedOutputNum+runningNum):
                # run
                simulator.run(idx)
                if NA_enable:
                    simulator.NA_adjust(NA)
                # save progress
                simulationResult["RawSampleNum"] = idx+1
                with open(simulationResultPath, "w") as f:
                    json.dump(simulationResult, f, indent=4)
            mean, CV = calculateR_CV.calculate_R_CV(
                sessionID, session, "mua_test.json")
            print(
                f"Session name: {sessionID} \n Reflectance mean: {mean} \nCV: {CV} ", end="\n\n")
            # remove file
            # remove_list = glob(os.path.join(
            #     config["OutputPath"], session, "mcx_output", "*.jdat"))
            # remove_list.sort(key=lambda x: int(x.split("_")[-2]))
            # remove_list = remove_list[1:]
            # for idx in range(len(remove_list)):
            #     os.remove(remove_list[idx])

        else:
            # run stage1 : run N sims to precalculate CV
            for idx in range(repeatTimes):
                # run
                simulator.run(idx)
                if NA_enable:
                    simulator.NA_adjust(NA)
                # save progress
                simulationResult["RawSampleNum"] = idx+1
                with open(simulationResultPath, "w") as f:
                    json.dump(simulationResult, f, indent=4)
            # calculate reflectance
            mean, CV = calculateR_CV.calculate_R_CV(
                sessionID, session, "mua_test.json")
            print(
                f"Session name: {sessionID} \n Reflectance mean: {mean} \nCV: {CV} \n Predict CV: {CV/np.sqrt(repeatTimes)}", end="\n\n")
            # run stage2 : run more sim to make up cvThreshold
            # reflectanceCV = {k: simulationResult["GroupingSampleCV"][k]
            #                  for k in simulationResult["GroupingSampleCV"]}

            predict_CV = max(CV)/np.sqrt(repeatTimes)
            while (predict_CV > cvThreshold):
                needAddOutputNum = int(
                    np.ceil((max(CV)**2)/(cvThreshold**2)) - repeatTimes)
                if needAddOutputNum > 0:
                    for idx in range(repeatTimes, repeatTimes+needAddOutputNum):
                        # run
                        simulator.run(idx)
                        if NA_enable:
                            simulator.NA_adjust(NA)
                        # save progress
                        simulationResult["RawSampleNum"] = idx+1
                        with open(simulationResultPath, "w") as f:
                            json.dump(simulationResult, f, indent=4)
                    # calculate reflectance
                    mean, CV = calculateR_CV.calculate_R_CV(
                        sessionID, session, "mua_test.json")
                    print(
                        f"Session name: {sessionID} \n Reflectance mean: {mean} \nCV: {CV} \n Predict CV: {CV/np.sqrt(repeatTimes+needAddOutputNum)}", end="\n\n")
                    predict_CV = max(CV)/np.sqrt(repeatTimes+needAddOutputNum)
                    repeatTimes = repeatTimes + needAddOutputNum
                    
            with open(simulationResultPath) as f:
                simulationResult = json.load(f)
            simulationResult['elapsed time'] = time.time() - now
            with open(simulationResultPath, "w") as f:
                json.dump(simulationResult, f, indent=4)

        print('ETA:{}/{}'.format(timer.measure(),
                                 timer.measure(run_idx / mus_set.shape[0])))
        sleep(0.01)

In [9]:
if __name__ == "__main__":

    # script setting
    # photon 1e9 take 1TB  CV 0.29%~0.81%  13mins per mus
    # photon 3e8 take 350GB CV 0.48%~1.08% 4mins per mus  wmc 110 mins per mus
    # -------------------------------------
    # photon 3e8 take 7mins for 10sims 24MB per file
    # photon 1e9 82MB per file 23mins

    # ID = sys.argv[1] #ID = "ctchen_ijv_small_to_large"
    # mus_start = int(sys.argv[2])
    # mus_end = int(sys.argv[3])

    result_folder = "kb"
    subject = "kb"
    ijv_type = 'large_to_small'
    mus_start = 1
    mus_end = 20
    NA_enable = 0  # 0 no NA, 1 consider NA
    NA = 0.22
    runningNum = 0  # (Integer or False)self.session
    cvThreshold = 100
    repeatTimes = 3
    
    run_mcx(result_folder=result_folder, 
            mus_type='high', 
            subject=subject, mus_start=mus_start, mus_end=mus_end, 
            NA_enable=NA_enable, NA=NA,
            runningNum=runningNum, cvThreshold=cvThreshold, 
            repeatTimes=repeatTimes, ijv_type=ijv_type)
    run_mcx(result_folder=result_folder, 
            mus_type='medium', 
            subject=subject, mus_start=mus_start, mus_end=mus_end, 
            NA_enable=NA_enable, NA=NA,
            runningNum=runningNum, cvThreshold=cvThreshold, 
            repeatTimes=repeatTimes, ijv_type=ijv_type)
    run_mcx(result_folder=result_folder, 
            mus_type='low', 
            subject=subject, mus_start=mus_start, mus_end=mus_end, 
            NA_enable=NA_enable, NA=NA,
            runningNum=runningNum, cvThreshold=cvThreshold, 
            repeatTimes=repeatTimes, ijv_type=ijv_type)
    
    result_folder = "kb"
    subject = "kb"
    ijv_type = 'small_to_large'
    mus_start = 1
    mus_end = 20
    NA_enable = 0  # 0 no NA, 1 consider NA
    NA = 0.22
    runningNum = 0  # (Integer or False)self.session
    cvThreshold = 100
    repeatTimes = 3
    
    run_mcx(result_folder=result_folder, 
            mus_type='high', 
            subject=subject, mus_start=mus_start, mus_end=mus_end, 
            NA_enable=NA_enable, NA=NA,
            runningNum=runningNum, cvThreshold=cvThreshold, 
            repeatTimes=repeatTimes, ijv_type=ijv_type)
    run_mcx(result_folder=result_folder, 
            mus_type='medium', 
            subject=subject, mus_start=mus_start, mus_end=mus_end, 
            NA_enable=NA_enable, NA=NA,
            runningNum=runningNum, cvThreshold=cvThreshold, 
            repeatTimes=repeatTimes, ijv_type=ijv_type)
    run_mcx(result_folder=result_folder, 
            mus_type='low', 
            subject=subject, mus_start=mus_start, mus_end=mus_end, 
            NA_enable=NA_enable, NA=NA,
            runningNum=runningNum, cvThreshold=cvThreshold, 
            repeatTimes=repeatTimes, ijv_type=ijv_type)

  0%|          | 0/20 [00:00<?, ?it/s]

∎∎ Start to run # 0 ... Finished !! ∎∎

∎∎ Start to run # 1 ... Finished !! ∎∎

∎∎ Start to run # 2 ... Finished !! ∎∎



  5%|▌         | 1/20 [00:30<09:46, 30.85s/it]

Session name: result\kb\kb_ijv_large_to_small\high\run_1 
 Reflectance mean: [2.05033283e-07 1.49451955e-07 1.70163094e-07 1.54184249e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.66982985e-08
 9.06671667e-09 7.50576961e-09 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00] 
CV: [ 4.14839415  6.12586322  7.95890251  4.64164632         nan         nan
         nan         nan         nan         nan         nan  6.25207822
 18.21277431 20.16627874         nan         nan         nan         nan
         nan         nan         nan] 
 Predict CV: [ 2.39507648  3.53676878  4.59507451  2.67985575         nan         nan
         nan         nan         nan         nan         nan  3.60963904
 10.51515015 11.64300646         nan         nan         nan         nan
         nan         nan         nan]

ETA:30s/10m
∎∎ Start to run # 0 ... Finished !! ∎∎

∎∎ Start to run

 10%|█         | 2/20 [00:56<08:16, 27.57s/it]

...

Session name: result\kb\kb_ijv_small_to_large\low\run_18 
 Reflectance mean: [7.85023144e-07 7.11375235e-07 7.05179451e-07 6.43158510e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.27830913e-08
 4.31504941e-08 5.51644128e-08 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00] 
CV: [3.65779968 2.17030706 3.07403839 4.61139911        nan        nan
        nan        nan        nan        nan        nan 6.26294632
 9.48006982 2.02310442        nan        nan        nan        nan
        nan        nan        nan] 
 Predict CV: [2.11183163 1.25302736 1.77479689 2.66239252        nan        nan
        nan        nan        nan        nan        nan 3.61591374
 5.47332086 1.16803988        nan        nan        nan        nan
        nan        nan        nan]

ETA:7m/8m
∎∎ Start to run # 0 ... Finished !! ∎∎

∎∎ Start to run # 1 ... Finished !! ∎∎

∎∎ Start to run # 2

 95%|█████████▌| 19/20 [07:17<00:25, 25.52s/it]

Session name: result\kb\kb_ijv_small_to_large\low\run_19 
 Reflectance mean: [8.22967044e-07 7.00892380e-07 6.74425489e-07 6.92976626e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.47055164e-08
 4.03475139e-08 4.86905047e-08 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00] 
CV: [ 2.86812146  3.47680023  0.96310356  1.90562289         nan         nan
         nan         nan         nan         nan         nan  9.63299852
  5.46329447 10.51887285         nan         nan         nan         nan
         nan         nan         nan] 
 Predict CV: [1.6559107  2.00733155 0.5560481  1.10021189        nan        nan
        nan        nan        nan        nan        nan 5.56161429
 3.15423453 6.07307407        nan        nan        nan        nan
        nan        nan        nan]

ETA:7m/8m
∎∎ Start to run # 0 ... Finished !! ∎∎

∎∎ Start to run # 1 ... Finished !! ∎∎

100%|██████████| 20/20 [07:40<00:00, 23.03s/it]

Session name: result\kb\kb_ijv_small_to_large\low\run_20 
 Reflectance mean: [8.17546747e-07 7.01525348e-07 7.10836361e-07 6.75576168e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.58381262e-08
 4.90576515e-08 5.71789550e-08 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00] 
CV: [2.29109989 2.74329392 3.24691278 4.22871973        nan        nan
        nan        nan        nan        nan        nan 5.2481191
 7.92430546 7.88571258        nan        nan        nan        nan
        nan        nan        nan] 
 Predict CV: [1.32276714 1.58384148 1.87460597 2.44145247        nan        nan
        nan        nan        nan        nan        nan 3.03000297
 4.57509989 4.55281828        nan        nan        nan        nan
        nan        nan        nan]

ETA:8m/8m
